In [ ]:
import pandas as pd
import google.generativeai as genai
from tqdm import tqdm
import time
from google.colab import userdata
from tenacity import retry, stop_after_attempt, wait_exponential  # NEW

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Configure Gemini API
def configure_gemini(api_key):
    """Configure the Gemini API with your API key."""
    genai.configure(api_key=api_key)
    return genai.GenerativeModel('gemini-1.5-flash')


In [ ]:
# Label generation prompt template
PROMPT_TEMPLATE = """
Please analyze the following academic paper and generate a unique, descriptive label that captures its main focus.
The label should be concise (3-5 words max) and suitable for use in a recommender system.

Paper Title: {title}

Abstract: {abstract}

Please provide only the label, nothing else. The label should be in the following format:
"label: [your generated label here]"
"""

In [ ]:

def extract_label_from_response(response_text):
    """Extract the label from Gemini's response."""
    if "label:" in response_text.lower():
        return response_text.split("label:")[1].strip()
    return response_text.strip()


def generate_paper_label(model, title, abstract, max_retries=4):
    """Generate a label for a paper using Gemini."""
    # Handle empty titles
    if pd.isna(title) or str(title).strip() == "":
        title = "Untitled Paper"

    # Handle empty abstracts
    if pd.isna(abstract) or str(abstract).strip() == "":
        abstract = "No abstract available"

    prompt = PROMPT_TEMPLATE.format(title=title, abstract=abstract)

    for attempt in range(max_retries):
        try:
            response = model.generate_content(prompt)
            label = extract_label_from_response(response.text)
            return label
        except Exception as e:
            print(f"Attempt {attempt + 1} failed for paper '{title[:50]}...': {str(e)}")
            if attempt < max_retries - 1:
                time.sleep(15)  # Wait before retrying
            else:
                print(f"Failed to generate label for paper '{title[:50]}...' after {max_retries} attempts")
                return None



## Label Papers function

In [ ]:
def label_papers(dataframe, api_key, num_papers=2500, save_interval=100):
    """
    Improved version that:
    1. Skips already labeled papers
    2. Has better rate limiting
    3. Can resume interrupted runs
    """
    # Load partial results if exists
    try:
        df = pd.read_csv('/content/drive/MyDrive/5000/labeled_papers_partial.csv')
        print("Resuming from partial progress...")
    except:
        df = dataframe.copy()
        df['label'] = None  # Initialize if new run

    model = configure_gemini(api_key)

    # Track progress
    unlabeled = df['label'].isna()
    to_process = min(num_papers, unlabeled.sum())
    print(f"Found {to_process} unlabeled papers to process...")

    processed_count = 0
    for idx in tqdm(df.index[:num_papers], total=num_papers):
        if pd.isna(df.at[idx, 'label']):
            label = generate_paper_label(model, df.at[idx, 'title'], df.at[idx, 'abstract'])
            df.at[idx, 'label'] = label
            processed_count += 1

            # Save progress periodically
            if processed_count % save_interval == 0:
                df.to_csv('/content/drive/MyDrive/5000/labeled_papers_partial.csv', index=False)
                print(f"\nSaved progress after {processed_count} papers")

    # Final save
    df.to_csv('/content/drive/MyDrive/5000/labeled_papers_final.csv', index=False)
    return df

In [ ]:
# Main execution
if __name__ == "__main__":
    # Load your CSV file
    csv_path = '/content/drive/MyDrive/5000/labeled_papers_first_2500_COMPLETED_3.csv'
    df = pd.read_csv(csv_path)

    # Get API key from Colab secrets
    try:
        GEMINI_API_KEY = userdata.get('Google-api-key-3')
    except Exception as e:
        raise ValueError("Please set up your Gemini API key in Colab secrets first") from e

    # Label first 2500 papers
    labeled_df = label_papers(df, GEMINI_API_KEY, num_papers=2500)
    # Count how many papers have a non-empty label
    num_labeled = labeled_df['label'][:2500].notna().sum()

    print(f"Successfully labeled {num_labeled}/{len(labeled_df)} papers.")
    # Save results
    output_path = '/content/drive/MyDrive/5000/labeled_papers_first_2500_COMPLETED_4.csv'
    labeled_df.to_csv(output_path, index=False)
    print(f"Results saved to {output_path}")

    # Show sample of results
    print("\nSample of labeled papers:")
    print(labeled_df[['title', 'label']].head())

Resuming from partial progress...
Found 2500 unlabeled papers to process...


 91%|█████████ | 2275/2500 [00:06<00:00, 228.67it/s]WARNING:tornado.access:429 POST /v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 103.80ms


Attempt 1 failed for paper 'Phonon generation by current-carrying nanostructur...': 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.


 91%|█████████▏| 2285/2500 [00:21<00:07, 27.05it/s] WARNING:tornado.access:429 POST /v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 103.02ms


Attempt 1 failed for paper 'Dynamical properties of low dimensional CuGeO$_3$ ...': 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.


...

Attempt 1 failed for paper 'Dynamical heterogeneities in a supercooled Lennard...': 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.


Attempt 2 failed for paper 'Dynamical heterogeneities in a supercooled Lennard...': 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.


Attempt 3 failed for paper 'Dynamical heterogeneities in a supercooled Lennard...': 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.


100%|██████████| 2500/2500 [04:13<00:00,  9.85it/s]


Successfully labeled 2500/5713 papers.
Results saved to /content/drive/MyDrive/5000/labeled_papers_first_2500_COMPLETED_4.csv

Sample of labeled papers:
                                               title  \
0  Genetic Algorithm for SU(2) Gauge Theory on a ...   
1         Restart Strategies and Internet Congestion   
2  Development and Evolution of Neural Networks i...   
3                                                NaN   
4  On the Signal Significance in the Presence of ...   

                                 label  
0              Lattice Gauge Theory GA  
1      Restart Strategies & Congestion  
2             Evolving Neural Networks  
3           Digital Organism Evolution  
4  Signal Significance & Uncertainties  


In [ ]:
# Main execution
if __name__ == "__main__":
    # Load your CSV file
    csv_path = '/content/drive/MyDrive/5000/labeled_papers_first_2500_COMPLETED_2.csv'
    df = pd.read_csv(csv_path)

    # Get API key from Colab secrets
    try:
        GEMINI_API_KEY = userdata.get('Google-api-key-3')
    except Exception as e:
        raise ValueError("Please set up your Gemini API key in Colab secrets first") from e

    # Label first 2500 papers
    labeled_df = label_papers(df, GEMINI_API_KEY, num_papers=2500)
    # Count how many papers have a non-empty label
    num_labeled = labeled_df['label'][:2500].notna().sum()

    print(f"Successfully labeled {num_labeled}/{len(labeled_df)} papers.")
    # Save results
    output_path = '/content/drive/MyDrive/5000/labeled_papers_first_2500_COMPLETED_3.csv'
    labeled_df.to_csv(output_path, index=False)
    print(f"Results saved to {output_path}")

    # Show sample of results
    print("\nSample of labeled papers:")
    print(labeled_df[['title', 'label']].head())

Resuming from partial progress...
Found 2500 unlabeled papers to process...


 71%|███████   | 1772/2500 [00:06<00:02, 253.39it/s]WARNING:tornado.access:429 POST /v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 127.80ms


Attempt 1 failed for paper '\mbox{$\pi$ excitation...': 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.


 71%|███████   | 1772/2500 [00:19<00:02, 253.39it/s]WARNING:tornado.access:429 POST /v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 203.50ms


Attempt 2 failed for paper '\mbox{$\pi$ excitation...': 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.


...

Attempt 2 failed for paper 'Dynamical heterogeneities in a supercooled Lennard...': 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.


100%|█████████▉| 2492/2500 [37:46<01:16,  9.51s/it]WARNING:tornado.access:429 POST /v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 178.22ms


Attempt 3 failed for paper 'Dynamical heterogeneities in a supercooled Lennard...': 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.
Failed to generate label for paper 'Dynamical heterogeneities in a supercooled Lennard...' after 3 attempts
Attempt 1 failed for paper 'Untitled Paper...': 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.


100%|██████████| 2500/2500 [38:05<00:00,  1.09it/s]


Successfully labeled 2475/5713 papers.
Results saved to /content/drive/MyDrive/5000/labeled_papers_first_2500_COMPLETED_2.csv

Sample of labeled papers:
                                               title  \
0  Genetic Algorithm for SU(2) Gauge Theory on a ...   
1         Restart Strategies and Internet Congestion   
2  Development and Evolution of Neural Networks i...   
3                                                NaN   
4  On the Signal Significance in the Presence of ...   

                                 label  
0              Lattice Gauge Theory GA  
1      Restart Strategies & Congestion  
2             Evolving Neural Networks  
3           Digital Organism Evolution  
4  Signal Significance & Uncertainties  
